In [1]:
pip install surprise

     |████████████████████████████████| 11.8MB 206kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617629 sha256=c56accdbd0545e28312597e466f0a0a7e28cc103584e2eae4b2a4274734bdee5
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [55]:
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise import NMF, SlopeOne#NMF-Алгоритм совместной фильтрации, основанный на неотрицательной матричной факторизации, SlopeOne-Простой, но точный алгоритм совместной фильтрации.
from surprise import accuracy
from surprise import Reader

In [56]:
import pandas as pd
movies = pd.read_csv('movies.csv')
movies = movies[:1000]
ratings = pd.read_csv('ratings.csv')
ratings = ratings[:1000]

In [57]:
data = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
data.dropna(inplace=True)
data.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,4.0,1.113766e+09
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,5.0,9.488858e+08
2,2,Jumanji (1995),Adventure|Children|Fantasy,4.0,4.0,1.113767e+09
3,3,Grumpier Old Men (1995),Comedy|Romance,8.0,3.0,8.582693e+08
5,5,Father of the Bride Part II (1995),Comedy,4.0,2.0,1.123990e+09


In [58]:
dataset = pd.DataFrame({
    'title': data.title,
    'userId': data.userId,
    'rating': data.rating
})

In [59]:
reader= Reader(rating_scale=(0.5, 5.0))
data_surprise = Dataset.load_from_df(dataset, reader)

In [60]:
trainset, testset = train_test_split(data_surprise, test_size=0.2)

In [61]:
algo = NMF(n_factors=20, n_epochs=20)
algo.fit(trainset)
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 1.0289


1.0288551966903376

In [62]:
from surprise import KNNWithMeans
algo2 = KNNWithMeans(k=10)
algo2.fit(trainset)
test_pred = algo2.test(testset)
accuracy.rmse(test_pred)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9978


0.9978255310993295

In [63]:
algo.predict(uid=4.0, iid='Toy Story (1995)').est

3.4285714285714284

In [64]:
algo2.predict(uid=4.0, iid='Toy Story (1995)').est

3.4285714285714284

In [65]:
current_user_id = 4.0
user_movies = data[data.userId == current_user_id].title.unique()

scores = []
titles = []

for movie in data.title.unique():
    if movie in user_movies:
        continue
        
    scores.append(algo.predict(uid=current_user_id, iid=movie).est)
    titles.append(movie)

In [66]:
sorted(scores)[-10:]

[3.4285714285714284,
 3.4285714285714284,
 3.4285714285714284,
 3.4285714285714284,
 3.4285714285714284,
 3.4285714285714284,
 3.4285714285714284,
 3.4285714285714284,
 3.4285714285714284,
 3.4285714285714284]

In [67]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [68]:
movie_genres = [change_string(g) for g in movies.genres.values]

In [69]:
movie_genres[1]

'Adventure Children Fantasy'

In [70]:
pip install --upgrade scikit-learn

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.7/dist-packages (0.24.2)


In [73]:
from sklearn.neighbors import KDTree
from sklearn.neighbors import NearestNeighbors

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(movie_genres)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

tree  = KDTree(X_train_tfidf.toarray()) 

In [ ]:
test = change_string("Adventure|Comedy|Fantasy|Crime")

predict = count_vect.transform([test])
X_tfidf2 = tfidf_transformer.transform(predict)

dist, ind = tree.query(X_train_tfidf.toarray()[:1], k=3) 
print(ind) #ближайшие соседи
print(dist) #расстояние

In [85]:
movies.iloc[ind[0]]

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
553,558,"Pagemaster, The (1994)",Action|Adventure|Animation|Children|Fantasy
664,673,Space Jam (1996),Adventure|Animation|Children|Comedy|Fantasy|Sc...


In [87]:
data.sort_values('timestamp', inplace=True)

In [91]:
title_genres = {}

for index, row in movies.iterrows():
    title_genres[row.title] = row.genres

In [111]:
import numpy as np
def recommend_for_user(user_id):
    current_user_id = user_id
    user_movies = data[data.userId == current_user_id].title.unique()
    
    last_user_movie = user_movies[-1]
    
    movie_genres = title_genres[last_user_movie]
    
    movie_genres = change_string(movie_genres)

    predict = count_vect.transform([movie_genres])
    X_tfidf2 = tfidf_transformer.transform(predict)

    res = KDTree(X_tfidf2.toarray())
    
    movies_to_score = movies.iloc[ind[0]].title.values

    scores = []
    titles = []

    for movie in movies_to_score:
        if movie in user_movies:
            continue

        scores.append(algo.predict(uid=current_user_id, iid=movie).est)
        titles.append(movie)
        
    
    best_indexes = np.argsort(scores)[-10:]
    for i in reversed(best_indexes):
        print(titles[i], scores[i])

In [112]:
data[data.userId == 2.0].sort_values('rating')

,movieId,title,genres,userId,rating,timestamp
897,849,Escape from L.A. (1996),Action|Adventure|Sci-Fi|Thriller,2.0,3.5,1.192914e+09
187,170,Hackers (1995),Action|Adventure|Crime|Thriller,2.0,3.5,1.192914e+09


In [113]:
recommend_for_user(2.0)

Space Jam (1996) 3.4285714285714284
Pagemaster, The (1994) 3.4285714285714284
Toy Story (1995) 3.4285714285714284


In [47]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier 

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(movie_genres)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

estimators = [
    ('lr', LogisticRegression()),
    ('dt', DecisionTreeClassifier()),
    ('rf', RandomForestClassifier())
]


clasificator = StackingClassifier(
    estimators=estimators, 
    final_estimator=RandomForestClassifier(n_estimators=70, criterion='entropy', min_samples_leaf=2, class_weight='balanced')
)

In [116]:
clasificator.fit(X_train_tfidf)

TypeError: ignored